In [1]:
import pandas as pd

In this notebook, we examine the CSV file loaded below. It contains science-quality flare data and was copied from the Dropbox directory `ML-SEP/Dataset/Science-Quality Data`; Ke Hu recommended using it. We make certain useful changes to the dataset and then save it as a data frame.

Judging by the output below,

- the times need to be turned into `datetime`s
- the flare class and peak intensity need to be extracted from `fl_class`
- the NOAA active region numbers need to be made integers

In [2]:
flare_data = pd.read_csv("../flare_data/sci_20100101_20240721.csv")
print(flare_data.shape)
print(flare_data.dtypes)
flare_data.head()

(31281, 16)
start time        object
end time          object
peak time         object
fl_class          object
noaa_ar_5min     float64
noaa_ar_5s       float64
hg1              float64
hg2              float64
car1             float64
car2             float64
rtheta1          float64
rtheta2          float64
xy1              float64
xy2              float64
solar_p_angle    float64
solar_radius     float64
dtype: object


,start time,end time,peak time,fl_class,noaa_ar_5min,noaa_ar_5s,hg1,hg2,car1,car2,rtheta1,rtheta2,xy1,xy2,solar_p_angle,solar_radius
0,2010/1/1 6:02,2010/1/1 6:13,2010/1/1 6:09,B1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010/1/1 12:00,2010/1/1 12:19,2010/1/1 12:09,B2.7,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010/1/1 12:27,2010/1/1 13:09,2010/1/1 12:43,B3.3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010/1/1 15:58,2010/1/1 16:31,2010/1/1 16:20,B2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010/1/1 18:20,2010/1/1 18:31,2010/1/1 18:27,B1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extract Flare Classes and Peak Intensities

A flare classification should be a string like `"C2.3"`; the multiplier, e.g. `"2.3"`, should be greater than or equal to one.

In [3]:
fl_class_pattern = r"^[ABCMX][1-9][0-9]*\.[0-9]+$"

Not all `fl_class` strings match the pattern; some seem to have multipliers that are less than one. Ke said that in the original data, there are multipliers that are less than one.

In [4]:
fl_class_matches_pattern = flare_data["fl_class"].str.match(fl_class_pattern)
flare_data.loc[~fl_class_matches_pattern, "fl_class"]

333      C0.9
3012     C0.9
4203     M0.9
4303     C0.9
4911     M0.9
4917     C0.9
5509     C0.9
5775     C0.9
6415     C0.9
6863     M0.9
7434     C0.9
7482     C0.9
7560     C0.9
7655     C0.9
7799     C0.9
7981     M0.9
7984     C0.9
8178     C0.9
9117     C0.9
9796     C0.9
9933     C0.9
9958     C0.9
10049    C0.9
10069    C0.9
10361    C0.9
10485    C0.9
12546    C0.9
13216    C0.9
13355    C0.9
13701    M0.9
14018    X0.9
14079    M0.9
14968    C0.9
15093    C0.9
15192    C0.9
15306    C0.9
16181    C0.9
16270    C0.9
16373    C0.9
16655    C0.9
17217    C0.9
17708    C0.9
17843    C0.9
18254    C0.9
18331    C0.9
19726    C0.9
20692    B0.9
22176    C0.9
22556    C0.9
23575    C0.9
25292    M0.9
25877    C0.9
26916    C0.9
30046    M0.9
30139    M0.9
Name: fl_class, dtype: object

It seems that those strings should match the pattern below.

In [5]:
fl_class_pattern2 = r"^[ABCMX]0\.[0-9]+$"

They do match that pattern.

In [ ]:
fl_class_matches_pattern2 = flare_data["fl_class"].str.match(fl_class_pattern2)
flare_data.loc[~fl_class_matches_pattern & ~fl_class_matches_pattern2, "fl_class"]

Series([], Name: fl_class, dtype: object)

The function below extracts the peak intensity from a `fl_class` string, e.g., given `"C2.3"`, `2.3 * 1e-6` is returned.

In [7]:
def get_peak_intensity(fl_class: str) -> float:
    flare_class = fl_class[0]
    multiplier = float(fl_class[1:])
    powers = {"A": 1e-8, "B": 1e-7, "C": 1e-6, "M": 1e-5, "X": 1e-4}
    return multiplier * powers[flare_class]

In [8]:
flare_data.insert(4, "peak_intensity", flare_data["fl_class"].map(get_peak_intensity))

All extractions succeeded.

In [9]:
flare_data.loc[flare_data["peak_intensity"].isna(), "fl_class"]

Series([], Name: fl_class, dtype: object)

The range of peak intensities seems reasonable.

In [10]:
(flare_data["peak_intensity"].min(), flare_data["peak_intensity"].max())

(9e-08, 0.00146)

The flare classes in the flare classifications may be inaccurate, e.g., if the flare classification is `"C0.9"`, then the flare class is actually `"B"`, not `"C"`. Now that we have the peak intensities, we can use the function below to compute the correct flare classes.

In [11]:
def get_flare_class(peak_intensity: float) -> str:
    thresholds = [10 ** i for i in range(-4, -9, -1)]
    flare_classes = ["X", "M", "C", "B", "A"]
    for threshold, flare_class in zip(thresholds, flare_classes):
        if peak_intensity >= threshold:
            return flare_class
    return pd.NA

We compute the correct flare classes. We also make the `flare_class` column a string column so that if there are `pd.NA`'s in the column, they won't be changed to `None`s when saving the data frame.

In [12]:
flare_data.insert(4, "flare_class", flare_data["peak_intensity"].map(get_flare_class))
flare_data["flare_class"] = flare_data["flare_class"].astype("string")

The counts below look reasonable.

In [13]:
flare_data["flare_class"].value_counts(dropna=False)

flare_class
C    17826
B    11327
M     2009
X      118
A        1
Name: count, dtype: Int64

The `fl_class` column is now redundant, so we can drop it.

In [14]:
flare_data.drop(columns="fl_class", inplace=True)

## Make the NOAA Active Region Numbers Integers

We make the NOAA active region columns integer columns. We use a nullable data type since there are missing values in these columns.

In [15]:
flare_data["noaa_ar_5min"] = flare_data["noaa_ar_5min"].astype("Int64")
flare_data["noaa_ar_5s"] = flare_data["noaa_ar_5s"].astype("Int64")

For most flares, `noaa_ar_5min` is either missing or zero.

In [16]:
flare_data["noaa_ar_5min"].value_counts(dropna=False)

noaa_ar_5min
<NA>     9669
0        3447
13615     144
12297     123
12403     116
         ... 
13202       1
12352       1
11608       1
11159       1
11599       1
Name: count, Length: 1708, dtype: Int64

The same is true of `noaa_ar_5s`.

In [17]:
flare_data["noaa_ar_5s"].value_counts(dropna=False)

noaa_ar_5s
<NA>     10667
0         3244
13615      144
12297      117
12403      112
         ...  
12477        1
12467        1
12462        1
12464        1
12446        1
Name: count, Length: 1695, dtype: Int64

## Turn Times into `datetime`s

The times appear to have the format below.

In [18]:
time_pattern = r"^(19|20)[0-9]{2}/[0-9]+/[0-9]+ [0-9]+:[0-9]+$"

All the `start time`s have that format.

In [19]:
start_time_matches_pattern = ~flare_data["start time"].isna() & flare_data["start time"].str.match(time_pattern)
flare_data.loc[~start_time_matches_pattern, "start time"]

Series([], Name: start time, dtype: object)

Some `end time`s don't.

In [20]:
end_time_matches_pattern = ~flare_data["end time"].isna() & flare_data["end time"].str.match(time_pattern)
flare_data.loc[~end_time_matches_pattern, "end time"]

23       NaN
31       NaN
32       NaN
40       NaN
50       NaN
        ... 
31254    NaN
31258    NaN
31269    NaN
31274    NaN
31277    NaN
Name: end time, Length: 3987, dtype: object

All the `end time`s that don't are missing. Ke said that the `end time`s are missing in the original data.

In [21]:
flare_data.loc[~end_time_matches_pattern, "end time"].value_counts(dropna=False)

end time
NaN    3987
Name: count, dtype: int64

All the `peak time`s match the format above.

In [22]:
peak_time_matches_pattern = ~flare_data["peak time"].isna() & flare_data["peak time"].str.match(time_pattern)
flare_data.loc[~peak_time_matches_pattern, "peak time"]

Series([], Name: peak time, dtype: object)

We turn all times into `datetime`s.

In [23]:
flare_data["start time"] = pd.to_datetime(flare_data["start time"], errors="coerce", format="%Y/%m/%d %H:%M", utc=True)
flare_data["end time"] = pd.to_datetime(flare_data["end time"], errors="coerce", format="%Y/%m/%d %H:%M", utc=True)
flare_data["peak time"] = pd.to_datetime(flare_data["peak time"], errors="coerce", format="%Y/%m/%d %H:%M", utc=True)

All `start time`s were successfully converted.

In [24]:
flare_data.loc[flare_data["start time"].isna(), "start time"]

Series([], Name: start time, dtype: datetime64[ns, UTC])

Some `end time`s weren't successfully converted.

In [25]:
flare_data.loc[flare_data["end time"].isna(), "end time"]

23      NaT
31      NaT
32      NaT
40      NaT
50      NaT
         ..
31254   NaT
31258   NaT
31269   NaT
31274   NaT
31277   NaT
Name: end time, Length: 3987, dtype: datetime64[ns, UTC]

The conversion only failed for missing `end time`s.

In [26]:
flare_data.loc[flare_data["end time"].isna() & end_time_matches_pattern, "end time"]

Series([], Name: end time, dtype: datetime64[ns, UTC])

All `peak time`s were successfully converted.

In [27]:
flare_data.loc[flare_data["peak time"].isna(), "peak time"]

Series([], Name: peak time, dtype: datetime64[ns, UTC])

For each flare, the `start time` precedes the `peak time`.

In [28]:
flare_data[flare_data["start time"] > flare_data["peak time"]]

,start time,end time,peak time,flare_class,peak_intensity,noaa_ar_5min,noaa_ar_5s,hg1,hg2,car1,car2,rtheta1,rtheta2,xy1,xy2,solar_p_angle,solar_radius


For each flare with an `end time`, the `start time`, `peak time`, and `end time` are properly ordered.

In [29]:
flare_data[
    ~flare_data["end time"].isna() &
    ((flare_data["start time"] > flare_data["end time"]) | (flare_data["peak time"] > flare_data["end time"]))
]

,start time,end time,peak time,flare_class,peak_intensity,noaa_ar_5min,noaa_ar_5s,hg1,hg2,car1,car2,rtheta1,rtheta2,xy1,xy2,solar_p_angle,solar_radius


We save the final data frame.

In [30]:
flare_data.to_parquet("../flare_data/flare_data.parquet")